In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()


{}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [ ]:
col_names = ['ID', 'Entity', 'Sentiment', 'Tweets']
train_df = pd.read_csv('twitter_training.csv', names=col_names)
test_df = pd.read_csv('twitter_validation.csv', names=col_names)

In [ ]:
train_df.head()

,ID,Entity,Sentiment,Tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
test_df.head()

,ID,Entity,Sentiment,Tweets
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [ ]:
train_df.isnull().sum() #there are null values in the training set which should be dropped

ID             0
Entity         0
Sentiment      0
Tweets       686
dtype: int64

In [ ]:
train_df.dropna(subset=['Tweets'], inplace=True)

In [ ]:
df = pd.concat([train_df,test_df],axis=0)
df.head()

,ID,Entity,Sentiment,Tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
df['Tweets'] = df['Tweets'].astype(str)

In [ ]:
df['Sentiment'].value_counts()

Negative      22624
Positive      20932
Neutral       18393
Irrelevant    13047
Name: Sentiment, dtype: int64

In [ ]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for word in r:
        input_txt = re.sub(word, "", input_txt)
    return input_txt
df['clean_Tweets'] = np.vectorize(remove_pattern)(df['Tweets'], "@[\w]*")
df.head()

,ID,Entity,Sentiment,Tweets,clean_tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands 2 and i will murder ...


In [ ]:
df['clean_Tweets'] = df['clean_Tweets'].str.replace("[^a-zA-Z#]"," ")
df.head()

<ipython-input-55-9dcda8031cf6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['clean_Tweets'] = df['clean_Tweets'].str.replace("[^a-zA-Z#]"," ")


,ID,Entity,Sentiment,Tweets,clean_tweet,clean_Tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...,getting borderlands will murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,I am coming to the borders and I will kill you...,coming borders will kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...,getting borderlands will kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...,coming borderlands will murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands and i will murder ...,getting borderlands will murder


In [ ]:
df['clean_Tweets'] = df['clean_Tweets'].apply(lambda x:" ".join([w for w in x.split() if len(w)>3]))
df.head()

,ID,Entity,Sentiment,Tweets,clean_tweet,clean_Tweets
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,im getting on borderlands and i will murder yo...,getting borderlands will murder
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,I am coming to the borders and I will kill you...,coming borders will kill
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,im getting on borderlands and i will kill you ...,getting borderlands will kill
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,im coming on borderlands and i will murder you...,coming borderlands will murder
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,im getting on borderlands and i will murder ...,getting borderlands will murder


In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda sentence: [stemmer.stem(word) for word in sentence])
tokenized_tweet

0                        [get, borderland, will, murder]
1                             [come, border, will, kill]
2                          [get, borderland, will, kill]
3                       [come, borderland, will, murder]
4                        [get, borderland, will, murder]
                             ...                        
995    [toronto, art, cultur, capit, canada, wonder, ...
996    [thi, actual, good, move, bring, more, viewer,...
997    [today, suck, time, drink, wine, play, borderl...
998     [bought, fraction, microsoft, today, small, win]
999    [johnson, johnson, stop, sell, talc, babi, pow...
Name: clean_Tweets, Length: 74996, dtype: object

In [ ]:
tokenized_tweet = df['clean_Tweets'].apply(lambda x: x.split())
tokenized_tweet

0                   [getting, borderlands, will, murder]
1                          [coming, borders, will, kill]
2                     [getting, borderlands, will, kill]
3                    [coming, borderlands, will, murder]
4                   [getting, borderlands, will, murder]
                             ...                        
995    [Toronto, arts, culture, capital, Canada, wond...
996    [tHIS, ACTUALLY, GOOD, MOVE, BRING, MORE, VIEW...
997    [Today, sucked, time, drink, wine, play, borde...
998    [Bought, fraction, Microsoft, today, Small, wins]
999    [Johnson, Johnson, stop, selling, talc, baby, ...
Name: clean_Tweets, Length: 74996, dtype: object

In [ ]:
train_df['Sentiment'] = train_df['Sentiment'].replace('Irrelevant', 'Neutral')
test_df['Sentiment'] = test_df['Sentiment'].replace('Irrelevant', 'Neutral')

In [ ]:
sentiment_mapping = {
    "Positive": 0,
    "Negative": 1,
    "Neutral": 2,  # Treating Irrelevant as Neutral
}

In [ ]:
sentiment_counts = train_df['Sentiment'].value_counts().sort_index()

sentiment_labels = ['Negative', 'Neutral', 'Positive']
sentiment_colors = ['red', 'blue', 'green']

fig = go.Figure(data=[go.Pie(labels=sentiment_counts.index,
                             values=sentiment_counts.values,
                             textinfo='percent+value+label',
                             marker_colors=sentiment_colors,
                             textposition='auto',
                             hole=.3)])

fig.update_layout(
    title_text='Sentiment Distribution',
    template='plotly_white',
    xaxis=dict(
        title='Sources',
    ),
    yaxis=dict(title='Number of Posts in Twitter',
    )
)

fig.update_traces(marker_line_color='black',
                  marker_line_width=1.5,
                  opacity=0.8)

fig.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words="english")
bow = bow_vectorizer.fit_transform(df['clean_Tweets'])

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(bow, df['Sentiment'], random_state=42, test_size=0.25)

In [ ]:
def accuracy(pred, y_test):
    l=0
    for i,j in zip(pred, y_test):
        if i==j:
            l=l+1
    return l/len(pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(x_train, y_train)

# Making predictions on the seen data, Checking for overfitting step-I
preds=model.predict(x_train)
accuracy(preds,y_train)

0.9034081817696944

In [ ]:
pred = model.predict(x_test) # Making Predictions on Test, checking for overfitting step-II
accuracy(pred, y_test)

0.7561469945063737

In [ ]:
y_train = train_df["Sentiment"].map(sentiment_mapping)
y_test = test_df["Sentiment"].map(sentiment_mapping)


In [ ]:
# Train a Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
y_test_pred = clf.predict(X_test)

# Calculate accuracy on the test data
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Top-1 Classification Accuracy on Test Data:", test_accuracy)

Top-1 Classification Accuracy on Test Data: 0.853
